In [26]:
import math
import numpy as np
import torch.distributed as dist
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data
import yaml
from torch.cuda import amp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

import test  # import test.py to get mAP after each epoch
from models.yolo import Model
from utils.datasets import create_dataloader
from utils.general import (
    torch_distributed_zero_first, labels_to_class_weights, plot_labels, check_anchors, labels_to_image_weights,
    compute_loss, plot_images, fitness, strip_optimizer, plot_results, get_latest_run, check_dataset, check_file,
    check_git_status, check_img_size, increment_dir, print_mutation, plot_evolution, set_logging, init_seeds,
non_max_suppression)
from utils.google_utils import attempt_download
from utils.torch_utils import ModelEMA, select_device, intersect_dicts
from utils.torch_utils import select_device, time_synchronized

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

In [53]:
exp = 'runs/exp0'
opt_ = check_file(f'{exp}/opt.yaml')
weights = f'{exp}/weights/best.pt'
nc, names = (1, ['coolingTower'])
rank = -1
device = torch.device('cpu')
half = device.type != 'cpu' 
with open(opt_) as f:
    opt = yaml.load(f, Loader=yaml.FullLoader)
opt = AttrDict(opt)
    
    
#Other params
imgsz = opt['img_size'][0]
batch_size = 1# opt['batch_size']
conf_thres = 0.2
iou_thres = 0.6

with open(opt['hyp']) as f:
    hyp = yaml.load(f, Loader=yaml.FullLoader)  # load hyps
    if 'box' not in hyp:
        warn('Compatibility: %s missing "box" which was renamed from "giou" in %s' %
             (opt['hyp'], 'https://github.com/ultralytics/yolov5/pull/1120'))
        hyp['box'] = hyp.pop('giou')
pretrained = weights.endswith('.pt')
if pretrained:
    with torch_distributed_zero_first(rank):
        attempt_download(weights)  # download if not found locally
    ckpt = torch.load(weights, map_location=device)  # load checkpoint
    if hyp.get('anchors'):
        ckpt['model'].yaml['anchors'] = round(hyp['anchors'])  # force autoanchor
    model = Model(opt['cfg'] or ckpt['model'].yaml, ch=3, nc=nc).to(device)  # create
    exclude = ['anchor'] if opt['cfg'] or hyp.get('anchors') else []  # exclude keys
    state_dict = ckpt['model'].float().state_dict()  # to FP32
    state_dict = intersect_dicts(state_dict, model.state_dict(), exclude=exclude)  # intersect
    model.load_state_dict(state_dict, strict=False)  # load
else:
    model = Model(opt['cfg'], ch=3, nc=nc).to(device)  # create

iouv = torch.linspace(0.5, 0.95, 10).to(device)  # iou vector for mAP@0.5:0.95
niou = iouv.numel()

In [54]:
model.eval()
with open('data/coolingTowers128.yaml') as f:
    data = yaml.load(f, Loader=yaml.FullLoader)  # model dict
check_dataset(data)  # check

img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
_ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once
path = data['train']  # path to val/test images
dataloader = create_dataloader(path, imgsz, batch_size, model.stride.max(), opt,
                               hyp=None, augment=False, cache=False, pad=0.5, rect=True)[0]


Scanning labels ../datasets/coolingTowers128/labels/train.cache (471 found, 0 missing, 151 empty, 1 duplicate, for 622 images): 622it [00:00, 20550.76it/s]


In [55]:
s = ('%20s' + '%12s' * 6) % ('Class', 'Images', 'Targets', 'P', 'R', 'mAP@.5', 'mAP@.5:.95')
p, r, f1, mp, mr, map50, map, t0, t1 = 0., 0., 0., 0., 0., 0., 0., 0., 0.
loss = torch.zeros(3, device=device)
jdict, stats, ap, ap_class = [], [], [], []
seen = 0

for batch_i, (img, targets, paths, shapes) in enumerate(tqdm(dataloader, desc=s)):
    img = img.to(device, non_blocking=True)
    img = img.half() if half else img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    targets = targets.to(device)
    nb, _, height, width = img.shape  # batch size, channels, height, width
    whwh = torch.Tensor([width, height, width, height]).to(device)

    # Disable gradients
    with torch.no_grad():
        # Run model
        t = time_synchronized()
        inf_out, train_out = model(img, augment=True)  # inference and training outputs
        t0 += time_synchronized() - t

        # Compute loss
        
        # Run NMS
        t = time_synchronized()
        output = non_max_suppression(inf_out, conf_thres=conf_thres, iou_thres=iou_thres)
        t1 += time_synchronized() - t

    # Statistics per image
    for si, pred in enumerate(output):
        labels = targets[targets[:, 0] == si, 1:]
        nl = len(labels)
        tcls = labels[:, 0].tolist() if nl else []  # target class
        seen += 1

        if pred is None:
            if nl:
                stats.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
            continue

        # Append to text file
        if save_txt:
            gn = torch.tensor(shapes[si][0])[[1, 0, 1, 0]]  # normalization gain whwh
            x = pred.clone()
            x[:, :4] = scale_coords(img[si].shape[1:], x[:, :4], shapes[si][0], shapes[si][1])  # to original
            for *xyxy, conf, cls in x:
                xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                line = (cls, conf, *xywh) if save_conf else (cls, *xywh)  # label format
                with open(str(out / Path(paths[si]).stem) + '.txt', 'a') as f:
                    f.write(('%g ' * len(line) + '\n') % line)

        # Clip boxes to image bounds
        clip_coords(pred, (height, width))
    break

               Class      Images     Targets           P           R      mAP@.5  mAP@.5:.95:   0%|          | 0/622 [00:02<?, ?it/s]


NameError: name 'save_txt' is not defined

In [62]:
pred

tensor([[3.56974e+02, 3.89185e+02, 4.94372e+02, 5.19322e+02, 3.43784e-01, 0.00000e+00]])